In [2]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import joblib
from datetime import datetime
import concurrent

BASE_DIR = "../"
PASQUINIS_PATH = BASE_DIR + "traces-netsoft-2017"
DATE = datetime.now().isoformat(timespec='seconds')
BASE_RESULTS_PATH = f'./resultados_pre_pesquisa/{DATE}'
MODELS_DIR = "models/"
MODELS_PATH_PREFIX = f'models/{DATE}_'

TRACES=[
    #"KV-BothApps-FlashcrowdLoad",
    #"KV-BothApps-PeriodicLoad",
    #"KV-SingleApp-FlashcrowdLoad",
    #"KV-SingleApp-PeriodicLoad",
    #"VoD-BothApps-FlashcrowdLoad",
    #"VoD-BothApps-PeriodicLoad",
    #"VoD-SingleApp-FlashcrowdLoad",
    "VoD-SingleApp-PeriodicLoad"]

NROWS = None
PERSIST = False
Y_METRIC = 'DispFrames'
VOD_SINGLEAPP_PERIODIC_LOAD_PATH = f'{PASQUINIS_PATH}/VoD-SingleApp-PeriodicLoad'

def nmae(y_pred, y_test):
    return abs(y_pred - y_test).mean() / y_test.mean()

if PERSIST:
    results_path = f'{BASE_RESULTS_PATH}'
    try:
        os.makedirs(results_path)
        os.makedirs(MODELS_DIR)
    except FileExistsError:
        print("Já criado diretório...")

In [3]:
y_dataset = pd.read_csv(f'{VOD_SINGLEAPP_PERIODIC_LOAD_PATH}/Y.csv', header=0, nrows=NROWS, index_col=0, usecols=['TimeStamp', 'DispFrames'], low_memory=True).apply(pd.to_numeric, errors='coerce').fillna(0)

x_files = ['X_cluster.csv', 'X_flow.csv', 'X_port.csv']

x_trace = pd.DataFrame()

for x_file in x_files:
    read_dataset = pd.read_csv(f'{VOD_SINGLEAPP_PERIODIC_LOAD_PATH}/{x_file}', 
                              header=0, index_col=0, low_memory=True, nrows=NROWS).apply(pd.to_numeric, errors='coerce').fillna(0)
    if len(x_trace.columns) != 0:
        x_trace.merge(read_dataset, how="inner",
            on="TimeStamp", copy=False)
    else:
        x_trace = read_dataset



In [12]:
X_train, X_test, y_train, y_test = train_test_split(x_trace, y_dataset, test_size=0.7, random_state=42)


In [8]:
import time

In [21]:

regression_tree = DecisionTreeRegressor() 

tempo_reg_tree = time.time()
regression_tree.fit(X_train, y_train)
tempo_reg_tree = time.time() - tempo_reg_tree

if PERSIST:
    joblib.dump(regression_tree, f'{MODELS_PATH_PREFIX}_model_regression-tree.sav')

print(f'Training Time Reg Tree: {tempo_reg_tree}')
pred_reg_tree = regression_tree.predict(X_test)
print(f'NMAE: {nmae(pred_reg_tree, y_test["DispFrames"])}')

Training Time Reg Tree: 32.23722505569458
NMAE: 0.10112599697648961


In [25]:
regr_random_forest = RandomForestRegressor(n_estimators=120, random_state=42, n_jobs=-1)

tempo_random_forest = time.time()
regr_random_forest.fit(X_train, y_train)
tempo_random_forest = time.time() - tempo_random_forest

if PERSIST:
    joblib.dump(regr_random_forest, f'{MODELS_PATH_PREFIX}_model_random-forest.sav')

print(f'Training Time Random Forest: {tempo_random_forest}')
pred_random_forest = regr_random_forest.predict(X_test)
print(f'NMAE: {nmae(pred_random_forest, y_test["DispFrames"])}')

/home/davi/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Training Time Random Forest: 320.60004210472107


In [26]:
print(f'NMAE: {nmae(pred_random_forest, y_test["DispFrames"])}')

NMAE: 0.0934177251849149


In [28]:
## para X_flow ou x_port

x_file = 'X_flow.csv' 

print(f'Only dataset {x_file}')
x_trace_minimal_flow = pd.DataFrame()

x_trace_minimal_flow = pd.read_csv(f'{VOD_SINGLEAPP_PERIODIC_LOAD_PATH}/{x_file}', 
                          header=0, index_col=0, low_memory=True, nrows=NROWS).apply(pd.to_numeric, errors='coerce').fillna(0)

X_train_flow, X_test_flow, y_train_flow, y_test_flow = train_test_split(x_trace_minimal_flow, y_dataset, test_size=0.7, random_state=42)

regression_tree_flow = DecisionTreeRegressor() 

tempo_reg_tree_flow = time.time()
regression_tree_flow.fit(X_train_flow, y_train_flow)
tempo_reg_tree_flow = time.time() - tempo_reg_tree_flow

if PERSIST:
    joblib.dump(regression_tree_flow, f'{MODELS_PATH_PREFIX}_model_regression-tree_for-{x_file}.sav')

print(f'Training Time only {x_file} Reg Tree: {tempo_reg_tree_flow}')
pred_reg_tree_flow = regression_tree_flow.predict(X_test_flow)
print(f'NMAE only {x_file}: {nmae(pred_reg_tree_flow, y_test["DispFrames"])}')

regr_random_forest_flow = RandomForestRegressor(n_estimators=120, random_state=42, n_jobs=-1)

tempo_random_forest_flow = time.time()
regr_random_forest_flow.fit(X_train_flow, y_train_flow)
tempo_random_forest_flow = time.time() - tempo_random_forest_flow

if PERSIST:
    joblib.dump(regr_random_forest_flow, f'{MODELS_PATH_PREFIX}_model_random-forest_for-{x_file}.sav')

print(f'Training Time Random Forest only {x_file}: {tempo_random_forest_flow}')
pred_random_forest_flow = regr_random_forest_flow.predict(X_test_flow)
print(f'NMAE only {x_file}: {nmae(pred_random_forest_flow, y_test["DispFrames"])}')



Only dataset X_flow.csv
Training Time only X_flow.csv Reg Tree: 0.3066692352294922
NMAE only X_flow.csv: 0.11325286084708491


/home/davi/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Training Time Random Forest only X_flow.csv: 3.2016818523406982
NMAE only X_flow.csv: 0.10733201366994324


In [29]:
#'X_port.csv'

x_file = 'X_port.csv' 

print(f'Only dataset {x_file}')
x_trace_minimal_port = pd.DataFrame()

x_trace_minimal_port = pd.read_csv(f'{VOD_SINGLEAPP_PERIODIC_LOAD_PATH}/{x_file}', 
                          header=0, index_col=0, low_memory=True, nrows=NROWS).apply(pd.to_numeric, errors='coerce').fillna(0)

X_train_port, X_test_port, y_train_port, y_test_port = train_test_split(x_trace_minimal_port, y_dataset, test_size=0.7, random_state=42)

regression_tree_port = DecisionTreeRegressor() 

tempo_reg_tree_port = time.time()
regression_tree_port.fit(X_train_port, y_train_port)
tempo_reg_tree_port = time.time() - tempo_reg_tree_port

if PERSIST:
    joblib.dump(regression_tree_port, f'{MODELS_PATH_PREFIX}_model_regression-tree_for-{x_file}.sav')

print(f'Training Time only {x_file} Reg Tree: {tempo_reg_tree_port}')
pred_reg_tree_port = regression_tree_port.predict(X_test_port)
print(f'NMAE only {x_file}: {nmae(pred_reg_tree_port, y_test["DispFrames"])}')

regr_random_forest_port = RandomForestRegressor(n_estimators=120, random_state=42, n_jobs=-1)

tempo_random_forest_port = time.time()
regr_random_forest_port.fit(X_train_port, y_train_port)
tempo_random_forest_port = time.time() - tempo_random_forest_port

if PERSIST:
    joblib.dump(regr_random_forest_port, f'{MODELS_PATH_PREFIX}_model_random-forest_for-{x_file}.sav')

print(f'Training Time Random Forest only {x_file}: {tempo_random_forest_port}')
pred_random_forest_port = regr_random_forest_port.predict(X_test_port)
print(f'NMAE only {x_file}: {nmae(pred_random_forest_port, y_test["DispFrames"])}')



Only dataset X_port.csv
Training Time only X_port.csv Reg Tree: 2.628509044647217
NMAE only X_port.csv: 0.11026255886288207


/home/davi/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Training Time Random Forest only X_port.csv: 25.735234260559082
NMAE only X_port.csv: 0.10501668143668873


In [32]:
x_trace.describe()

,0_all_..usr,0_all_..nice,0_all_..sys,0_all_..iowait,0_all_..steal,0_all_..irq,0_all_..soft,0_all_..guest,0_all_..gnice,0_all_..idle,...,5_X.dev.mapper.cloud..5.root_Ifree,5_X.dev.mapper.cloud..5.root_Iused,5_X.dev.mapper.cloud..5.root_.Iused,5_X.dev.sda1_MBfsfree,5_X.dev.sda1_MBfsused,5_X.dev.sda1_.fsused,5_X.dev.sda1_.ufsused,5_X.dev.sda1_Ifree,5_X.dev.sda1_Iused,5_X.dev.sda1_.Iused
count,37036.000000,37036.0,37036.000000,37036.000000,37036.0,37036.000000,37036.000000,37036.0,37036.0,37036.000000,...,3.703600e+04,37036.000000,3.703600e+04,37036.0,37036.0,3.703600e+04,3.703600e+04,37036.0,37036.0,37036.00
mean,0.110153,0.0,0.205758,0.015503,0.0,0.000009,0.002603,0.0,0.0,99.666505,...,2.598523e+07,327478.000243,1.240000e+00,139.0,89.0,3.894000e+01,4.428000e+01,124184.0,312.0,0.25
std,0.074545,0.0,0.070880,0.022714,0.0,0.000588,0.010129,0.0,0.0,0.115412,...,1.873393e-02,0.018734,8.038123e-14,0.0,0.0,3.478154e-11,2.329901e-11,0.0,0.0,0.00
min,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,94.560000,...,2.598522e+07,327478.000000,1.240000e+00,139.0,89.0,3.894000e+01,4.428000e+01,124184.0,312.0,0.25
25%,0.080000,0.0,0.170000,0.000000,0.0,0.000000,0.000000,0.0,0.0,99.620000,...,2.598523e+07,327478.000000,1.240000e+00,139.0,89.0,3.894000e+01,4.428000e+01,124184.0,312.0,0.25
50%,0.080000,0.0,0.210000,0.000000,0.0,0.000000,0.000000,0.0,0.0,99.670000,...,2.598523e+07,327478.000000,1.240000e+00,139.0,89.0,3.894000e+01,4.428000e+01,124184.0,312.0,0.25
75%,0.130000,0.0,0.250000,0.040000,0.0,0.000000,0.000000,0.0,0.0,99.750000,...,2.598523e+07,327478.000000,1.240000e+00,139.0,89.0,3.894000e+01,4.428000e+01,124184.0,312.0,0.25
max,4.340000,0.0,3.220000,0.380000,0.0,0.040000,0.130000,0.0,0.0,99.960000,...,2.598523e+07,327480.000000,1.240000e+00,139.0,89.0,3.894000e+01,4.428000e+01,124184.0,312.0,0.25


In [33]:
# obs não é minimal de minimal com stepwise!
x_trace_minimal_port.describe()

,0_RxPacktes,0_TxPacktes,0_RxBytes,0_TxBytes,1_RxPacktes,1_TxPacktes,1_RxBytes,1_TxBytes,2_RxPacktes,2_TxPacktes,...,41_RxBytes,41_TxBytes,42_RxPacktes,42_TxPacktes,42_RxBytes,42_TxBytes,43_RxPacktes,43_TxPacktes,43_RxBytes,43_TxBytes
count,37036.000000,37036.000000,3.703600e+04,37036.000000,37036.000000,37036.000000,37036.000000,3.703600e+04,37036.0,37036.000000,...,37036.000000,3.703600e+04,37036.000000,37036.000000,3.703600e+04,37036.000000,37036.000000,37036.000000,37036.000000,3.703600e+04
mean,5436.947349,1482.370801,7.867343e+06,100573.688654,596.418026,2402.349795,40508.881467,5.569556e+06,0.0,0.166271,...,60077.391916,7.366081e+06,2401.622421,596.183632,3.466114e+06,40493.178718,596.111756,2401.672238,40488.784804,5.682019e+06
std,1368.943917,674.070781,1.853832e+06,45595.793627,314.414592,652.344742,21207.270619,1.504301e+06,0.0,0.543109,...,25178.909084,1.695046e+06,652.011836,314.317629,8.948487e+05,21201.210481,314.316178,651.952146,21201.178254,1.473328e+06
min,28.000000,6.000000,4.239200e+04,596.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,4309.000000,779.000000,6.403354e+06,53038.250000,277.000000,1917.000000,19007.500000,4.454762e+06,0.0,0.000000,...,34675.000000,6.102802e+06,1918.000000,277.000000,2.825678e+06,18998.500000,277.000000,1916.000000,19003.750000,4.619797e+06
50%,5626.000000,1566.000000,8.114068e+06,106377.000000,625.000000,2458.000000,42500.000000,5.710244e+06,0.0,0.000000,...,62606.000000,7.524756e+06,2455.000000,626.000000,3.537171e+06,42559.000000,626.000000,2456.000000,42558.500000,5.821844e+06
75%,6514.000000,2055.000000,9.308527e+06,139255.250000,855.000000,2885.000000,57839.000000,6.673491e+06,0.0,0.000000,...,81094.250000,8.643120e+06,2883.000000,854.000000,4.115687e+06,57847.500000,854.000000,2883.000000,57825.250000,6.747885e+06
max,9416.000000,3319.000000,1.336043e+07,226086.000000,1478.000000,4476.000000,101225.000000,1.067704e+07,0.0,6.000000,...,126117.000000,1.267862e+07,4525.000000,1476.000000,6.581067e+06,100981.000000,1477.000000,4524.000000,101080.000000,1.088535e+07


In [34]:
# obs não é minimal de minimal com stepwise!
x_trace_minimal_flow.describe()

,0_RxPacktes,0_RxBytes,1_RxPacktes,1_RxBytes,2_RxPacktes,2_RxBytes,3_RxPacktes,3_RxBytes,4_RxPacktes,4_RxBytes,...,7_RxPacktes,7_RxBytes,8_RxPacktes,8_RxBytes,9_RxPacktes,9_RxBytes,10_RxPacktes,10_RxBytes,11_RxPacktes,11_RxBytes
count,37036.000000,37036.000000,37036.000000,3.703600e+04,37036.000000,37036.000000,37036.000000,3.703600e+04,37036.000000,37036.000000,...,37036.000000,3.703600e+04,37036.000000,37036.000000,37036.000000,3.703600e+04,37036.000000,37036.000000,37036.000000,3.703600e+04
mean,22.879199,1532.793957,22.904984,1.098690e+05,22.842775,1530.237931,23.360757,1.098034e+05,22.976131,1539.266308,...,23.467005,1.097233e+05,22.905794,1534.650853,23.013986,1.099182e+05,22.814883,1528.337887,23.215736,1.096529e+05
std,10.347252,708.601947,11.583598,5.817529e+04,10.126399,693.021732,11.554781,5.760927e+04,10.436408,712.908449,...,11.710848,5.770255e+04,10.404614,713.655725,11.580033,5.785379e+04,10.122568,692.288325,11.480318,5.747716e+04
min,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00
25%,16.000000,1056.000000,15.000000,7.224675e+04,16.000000,1068.000000,15.000000,7.312600e+04,16.000000,1056.000000,...,15.000000,7.306000e+04,16.000000,1056.000000,15.000000,7.210100e+04,16.000000,1068.000000,15.000000,7.312600e+04
50%,21.000000,1386.000000,20.000000,9.938800e+04,21.000000,1386.000000,21.000000,9.938150e+04,21.000000,1386.000000,...,21.000000,9.943850e+04,21.000000,1386.000000,20.000000,9.938800e+04,21.000000,1386.000000,21.000000,9.911000e+04
75%,27.000000,1848.000000,28.000000,1.357080e+05,27.000000,1818.000000,29.000000,1.346680e+05,28.000000,1848.000000,...,29.000000,1.347340e+05,28.000000,1848.000000,28.000000,1.355080e+05,27.000000,1830.000000,28.000000,1.346020e+05
max,227.000000,18934.000000,254.000000,2.409786e+06,278.000000,22556.000000,273.000000,2.311760e+06,216.000000,17840.000000,...,309.000000,2.319770e+06,243.000000,20094.000000,263.000000,2.397236e+06,278.000000,22556.000000,265.000000,2.311232e+06
